<a href="https://colab.research.google.com/github/sundarjhu/Astrostatistics2025/blob/main/Lesson19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dynesty ultranest arviz pytensor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.8 MB/s eta 0:00:00



# Frequentist vs Bayesian Model Selection: Quadratic vs Quintic Fits

This notebook demonstrates model selection using both **frequentist** (AIC/BIC) and **Bayesian** (marginal likelihood) methods.  
We generate synthetic data from a quadratic model and compare a quadratic and a quintic polynomial fit.

The Bayesian evidence is computed robustly using **nested sampling** (`dynesty`, `UltraNest`), while **PyMC's SMC** evidence is shown with caveats about its limitations.



## 1. Generate Synthetic Data


In [ ]:

import numpy as np, matplotlib.pyplot as plt

rng = np.random.default_rng(42)
x = np.linspace(-3, 3, 60)
y_true = 1 + 2*x - 0.5*x**2
yerr = np.ones_like(x)
y = y_true + rng.normal(0, yerr)

plt.errorbar(x, y, yerr, fmt='o', color='k', label='data')
plt.plot(x, y_true, 'r--', label='true model')
plt.xlabel('x'); plt.ylabel('y'); plt.legend()
plt.title('Synthetic data (quadratic truth)')
plt.show()



## 2. Frequentist Fits: Quadratic and Quintic

Compute AIC and BIC to compare models under frequentist assumptions.


In [ ]:

from scipy.optimize import curve_fit

def fit_poly(x, y, yerr, deg):
    p0 = np.zeros(deg+1)
    popt, pcov = curve_fit(lambda x,*p: np.polyval(p,x), x, y, sigma=yerr, p0=p0)
    return popt, pcov

def chi2(p):
    return np.sum(((y - np.polyval(p, x))/yerr)**2)

p2, c2 = fit_poly(x, y, yerr, 2)
p5, c5 = fit_poly(x, y, yerr, 5)

chi2_2, chi2_5 = chi2(p2), chi2(p5)
n = len(y)
k2, k5 = len(p2), len(p5)

AIC2, AIC5 = 2*k2 + chi2_2, 2*k5 + chi2_5
BIC2, BIC5 = k2*np.log(n) + chi2_2, k5*np.log(n) + chi2_5

print(f"AIC: quadratic={AIC2:.1f}, quintic={AIC5:.1f}")
print(f"BIC: quadratic={BIC2:.1f}, quintic={BIC5:.1f}")



## 3. Bayesian Evidence via Nested Sampling (dynesty)


In [ ]:

import dynesty

def loglike(theta):
    model = np.polyval(theta, x)
    return -0.5*np.sum(((y - model)/yerr)**2)

def prior_transform(u):
    return 20*(u - 0.5)

logZ_dynesty = {}
for deg in [2, 5]:
    ndim = deg + 1
    sampler = dynesty.NestedSampler(loglike, prior_transform, ndim, nlive=300)
    sampler.run_nested()
    res = sampler.results
    logZ_dynesty[deg] = res.logz[-1]
    print(f"dynesty logZ (deg {deg}) = {res.logz[-1]:.2f} ± {res.logzerr[-1]:.2f}")



## 4. Bayesian Evidence via UltraNest (quiet mode)


In [ ]:

import ultranest, logging, os, contextlib

def loglike_ultra(theta):
    model = np.polyval(theta, x)
    return -0.5*np.sum(((y - model)/yerr)**2)

def prior_ultra(u):
    return 20*(u - 0.5)

# Silence most output
logger = logging.getLogger("ultranest")
logger.setLevel(logging.WARNING)

logZ_ultra = {}
for deg in [2, 5]:
    names = [f"a{i}" for i in range(deg+1)]
    sampler = ultranest.ReactiveNestedSampler(names, loglike_ultra, transform=prior_ultra)
    with open(os.devnull, 'w') as f, contextlib.redirect_stderr(f):
        result = sampler.run(min_num_live_points=300, dlogz=0.1, show_status=False, viz_callback=False)
    logZ_ultra[deg] = result['logz']
    print(f"UltraNest logZ (deg {deg}) = {result['logz']:.2f} ± {result['logzerr']:.2f}")



## 5. Compare Evidences and Interpret


In [ ]:

logZs = {'dynesty': logZ_dynesty, 'ultranest': logZ_ultra}
for method, vals in logZs.items():
    if len(vals) == 2:
        lnB = vals[2] - vals[5]
        print(f"{method:8s}: DeltalnB = {lnB:.2f}")



## 7. Discussion
- Dynesty and UltraNest produce consistent evidence values.
- PyMC's evidence field is often NaN due to current API limitations.
- Nested sampling is the robust and recommended way to compute Bayesian evidence.

| Δln Z | Strength of Evidence |
|:------:|:--------------------|
| < 1 | Inconclusive |
| 1–2.5 | Weak |
| 2.5–5 | Moderate |
| > 5 | Strong |

In this dataset (quadratic truth), all samplers should favor the quadratic model.
